In [ ]:
pip install openai

In [ ]:
pip install "instructor"

In [17]:
# Setup
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import instructor
from pydantic import BaseModel, Field
from openai import AzureOpenAI
from enum import Enum
from typing import List

# Using User Managed Identity client id
# https://learn.microsoft.com/en-us/python/api/overview/azure/identity-readme?view=azure-python#managed-identity-support
client_id = "<client_id>"

# Azure Open AI endpoint URL
endpoint_url = "<endpoint_url>"

#### Simple set of Sample tickets

In [18]:
support_ticket_1 = """Subject: Unable to Access Work Email Account
Description:
Hi IT Team,
I’ve been trying to log into my work email account since this morning and I keep getting a “password incorrect” error. I’m 100% sure I’m using the correct password. I even reset it once, and the new password doesn’t work either. Could you please check if something is wrong with my account?
""".strip()

support_ticket_2 = """Subject: Error Message in Project Management Application
Description:
Hello Support,
Every time I open the Project Management desktop app, I get an error that says “Unable to connect to server.” I’ve tried restarting my computer, and my internet connection is fine. I also tested another coworker’s laptop using my login, and it worked there. It might be specific to my machine.""".strip()

support_ticket_3 = """Subject: Frequent VPN Disconnections
Description:
Hello,
My VPN connection drops randomly multiple times a day. It reconnects automatically, but it disrupts my remote desktop sessions. Is there an update or a known issue with the VPN? Could you look into possible network stability problems?"""

support_ticket_4 = """Subject: A Magic Story 
Description: A long, long time ago a little boy was walking through a park. In the middle of the park there was a tree with a sign on it. The sign said "I am a magic tree. Say the magic words and you will see."
The boy tried to guess the magic words. He tried abracadabra, supercalifragilisticexpialidocious, ta-ta-ra, and many more... but none of them worked."""

support_ticket_5 = """Subject: Het werkt allemaal niet....
Description: Het papier loopt weer vast. We kunnen zo echt ons werkt niet doen. Kom onmiddelijk helpen!!!!""" 

#### Using an Instructor response model

In [19]:
class TicketCategory(str, Enum):
    SOFTWARE_ISSUES = "Software Issues"
    ACCOUNT_ISSUES = "Account Issues"
    HARDWARE_ISSUES = "Hardware Issues"
    CONNECTIVITY_ISSUES = "Connectivity Issues"
    SECURITY_ISSUES = "Security Issues"
    DATA_ISSUES = "Data Issues"
    OTHER = "Other"

class CustomerSentiment(str, Enum):
    ANGRY = "Angry"
    FRUSTRATED = "Frustrated"
    NEUTRAL = "Neutral"
    SATISFIED = "Satisfied"
    UNKNOWN = "Unknown"

class TicketUrgency(str, Enum):
    LOW = "Low"
    MEDIUM = "Medium"
    HIGH = "High"
    CRITICAL = "Critical"
    UNKNOWN = "Unknown"

class TicketClassification(BaseModel):
    category: TicketCategory
    urgency: TicketUrgency
    sentiment: CustomerSentiment
    key_information: List[str] = Field(description="List of key points extracted from the ticket")
    suggested_action: str = Field(description="Brief suggestion for handling the ticket")

In [20]:
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(managed_identity_client_id=client_id), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    api_version="2024-08-01-preview",
    azure_endpoint=endpoint_url,
    azure_ad_token_provider=token_provider
)

In [21]:
# https://python.useinstructor.com/
# Patch with Instructor
client = instructor.from_openai(client)

In [22]:
def classify_ticket(ticket_text: str) -> TicketClassification:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=TicketClassification,
        temperature=0,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": "Analyze the following customer support ticket and extract the requested information."
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

In [25]:
result1 = classify_ticket(support_ticket_1)
result2 = classify_ticket(support_ticket_2)
result3 = classify_ticket(support_ticket_3)
result4 = classify_ticket(support_ticket_4)
result5 = classify_ticket(support_ticket_5)

print(result1.model_dump_json(indent=2))
print(result2.model_dump_json(indent=2))
print(result3.model_dump_json(indent=2))
print(result4.model_dump_json(indent=1))
print(result5.model_dump_json(indent=2))

{
  "category": "Account Issues",
  "urgency": "High",
  "sentiment": "Frustrated",
  "key_information": [
    "User unable to log into work email account",
    "Receiving 'password incorrect' error",
    "User is sure they are using the correct password",
    "User has reset the password but it still doesn't work"
  ],
  "suggested_action": "Investigate the user's account for any issues and assist with password reset if necessary."
}
{
  "category": "Connectivity Issues",
  "urgency": "Medium",
  "sentiment": "Frustrated",
  "key_information": [
    "Error message: 'Unable to connect to server'",
    "Restarted computer",
    "Internet connection is fine",
    "Tested on coworker's laptop with successful login",
    "Issue might be specific to the user's machine"
  ],
  "suggested_action": "Investigate the user's machine for connectivity issues or application settings."
}
{
  "category": "Connectivity Issues",
  "urgency": "Medium",
  "sentiment": "Frustrated",
  "key_information": [
